In [1]:
using Revise
using Pkg

ENV["PYTHON"] = Sys.which("python")
ENV["PYCALL_JL_RUNTIME_PYTHON"] = Sys.which("python")
Pkg.build("PyCall")
using FileIO
using JLD2
include("../src/RiskSensitiveSAC.jl")
using .RiskSensitiveSAC

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/51cab8e982c5b598eea9c8ceaced4b58d9dd37c9/build.log`


    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/9816a3826b0ebf49ab4926e2b18842ad8b5c8f04/build.log`


┌ Error: Unable to load the 'rospy' python package!
│ Has an environment setup script been run?
└ @ RobotOS /home/kh-ryu/.julia/packages/RobotOS/j0Tsl/src/RobotOS.jl:35


┌ Info: Number of Julia Thread(s): 1
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/RiskSensitiveSAC.jl:39
┌ Info: CUDA Device: NVIDIA GeForce RTX 3060
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/RiskSensitiveSAC.jl:40
┌ Info: Python executable used by PyCall: /home/kh-ryu/anaconda3/envs/trajectron++/bin/python
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/RiskSensitiveSAC.jl:41


# 1. Default

In [2]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

test_data_name = "hotel_test.pkl";                                                  # test data set name
test_scene_id = 0;                                                                  # test data id
start_time_idx = 401;                                                               # start time index in test data
ego_pos_init_vec = [-1.5, -8.5] .+ [-1.393743, 2.978962];                           # initial ego position [x, y] [m]
ego_pos_goal_vec = [3.5, 0.0]   .+ [-1.393743, 2.978962];                           # goal ego position [x, y] [m]
target_speed = 1.0;                                                                 # target speed [m/s]
sim_horizon = 10.0;       

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [3]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 ego_pos_init_vec=ego_pos_init_vec,
                 ego_pos_goal_vec=ego_pos_goal_vec,
                 target_speed=target_speed,
                 sim_horizon=sim_horizon,
                 verbose=true);

Scene Mode: data
Prediction Mode: trajectron
Deterministic Prediction: false
Loaded evaluation data from /home/kh-ryu/DRC_Project/DRCC-MPC/Trajectron-plus-plus/experiments/processed/eth_test.pkl
Looking at the eth_test.pkl sequence, data_id 0, start_idx 905


Loaded Trajectron model from /home/kh-ryu/DRC_Project/DRCC-MPC/Trajectron-plus-plus/experiments/pedestrians/models/eth_attention_radius_3/model_registrar-100.pt


In [4]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec,
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false);

┌ Warning: It's not recommended to use allowscalar([true]) to allow scalar indexing.
│ Instead, use `allowscalar() do end` or `@allowscalar` to denote exactly which operations can use scalar operations.
└ @ GPUArraysCore /home/kh-ryu/.julia/packages/GPUArraysCore/GMsgk/src/GPUArraysCore.jl:188


┌ Warning: Time 0.00 [sec]: SAC computation took 25.285 [sec], which exceeds the maximum computation time allowed.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/rs_sac_controller.jl:733


┌ Warning: Time 0.00 [sec]: This evaluation iteration took 0.12 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 0.10 [sec]: This evaluation iteration took 0.064 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 0.20 [sec]: SAC computation took 0.192 [sec], which exceeds the maximum computation time allowed.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/rs_sac_controller.jl:733
┌ Warning: Time 0.20 [sec]: This evaluation iteration took 0.193 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


...

┌ Warning: Time 8.40 [sec]: This evaluation iteration took 0.083 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 8.80 [sec]: This evaluation iteration took 0.073 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 9.20 [sec]: This evaluation iteration took 0.067 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 9.60 [sec]: This evaluation iteration took 0.069 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


Average computation time: 0.03699679085702607
std of computation time: 0.044480262945688465


In [ ]:
display_log(result.log)

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.02, fps=50, xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="2_1_data_trajectron.gif")

In [ ]:
save("2_1_data_trajectron.jld2", "result", result)

# 2. Risk Sensitive

In [5]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

σ_risk = 1.0;
prediction_rng_seed = 1;

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [6]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 ego_pos_init_vec=ego_pos_init_vec,
                 ego_pos_goal_vec=ego_pos_goal_vec,
                 target_speed=target_speed,
                 sim_horizon=sim_horizon,
                 verbose=true);

Scene Mode: data
Prediction Mode: trajectron
Deterministic Prediction: false
Loaded evaluation data from /home/kh-ryu/DRC_Project/DRCC-MPC/Trajectron-plus-plus/experiments/processed/eth_test.pkl
Looking at the eth_test.pkl sequence, data_id 0, start_idx 905
Loaded Trajectron model from /home/kh-ryu/DRC_Project/DRCC-MPC/Trajectron-plus-plus/experiments/pedestrians/models/eth_attention_radius_3/model_registrar-100.pt


In [7]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec,
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false);

┌ Warning: Time 0.00 [sec]: SAC computation took 0.166 [sec], which exceeds the maximum computation time allowed.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/rs_sac_controller.jl:733


┌ Warning: Time 0.10 [sec]: This evaluation iteration took 0.029 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 0.20 [sec]: This evaluation iteration took 0.025 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 0.30 [sec]: This evaluation iteration took 0.024 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 0.40 [sec]: This evaluation iteration took 0.08 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


...

┌ Warning: Time 8.40 [sec]: This evaluation iteration took 0.079 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 8.80 [sec]: This evaluation iteration took 0.062 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 9.20 [sec]: This evaluation iteration took 0.062 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


┌ Warning: Time 9.60 [sec]: This evaluation iteration took 0.06 [sec], which exceeds dtc.
└ @ Main.RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRCC-MPC/src/evaluation.jl:291


Average computation time: 0.03576384168682677
std of computation time: 0.049528247673661735


In [ ]:
display_log(result.log)

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.1, fps=5,xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -2.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="2_2_data_trajectron_risk_1.0.gif")

In [ ]:
# save("2_2_data_trajectron_risk_1.0.jld2", "result", result)

# 3. Deterministic (mode-mode) Prediction

In [ ]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

deterministic = true;
num_samples = 1;

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [ ]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 ego_pos_init_vec=ego_pos_init_vec,
                 ego_pos_goal_vec=ego_pos_goal_vec,
                 target_speed=target_speed,
                 sim_horizon=sim_horizon,
                 verbose=true);

In [ ]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec,
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false);

In [ ]:
display_log(result.log)

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.02, fps=50, xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="2_3_data_trajectron_deterministic.gif")

In [ ]:
save("2_3_data_trajectron_deterministic.jld2", "result", result)

# 4. Robot-Future-Conditional: PEDESTRIAN/236 Replaced with Ego

In [ ]:
using LinearAlgebra

include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

incl_robot_node = true;
use_robot_future = true;

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [ ]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 sim_horizon=sim_horizon,
                 ado_id_to_replace="PEDESTRIAN/236",
                 verbose=true);

In [ ]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, maximum(target_trajectory)[2],
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false,
                  ado_id_removed="PEDESTRIAN/236");

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.02, fps=50, xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="2_4_data_trajectron_future_conditional.gif",
         show_nominal_trajectory=true)

In [ ]:
save("2_4_data_trajectron_future_conditional.jld2", "result", result)

# 5. No Robot-Future-Conditional: PEDESTRIAN/236 Replaced with Ego

In [ ]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");


include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [ ]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 sim_horizon=sim_horizon,
                 ado_id_to_replace="PEDESTRIAN/236",
                 verbose=true);

In [ ]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, maximum(target_trajectory)[2],
                  target_speed, measurement_schedule, target_trajectory,
                  pos_error_replan, nominal_control=false,
                  ado_id_removed="PEDESTRIAN/236");

In [ ]:
display_log(result.log)

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.02, fps=50,  xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="2_5_data_trajectron_no_future_conditional.gif")

In [ ]:
save("2_5_data_trajectron_future_conditional.jld2", "result", result)